# DataFrames: Read and Write Data
     
Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  In this example we read and write data with the popular CSV and Parquet formats, and discuss best practices when using these formats.

In [1]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/usr/share/miniconda3/envs/dask-examples/lib/python3.8/site-packages/IPython/core/display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [2]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://10.1.0.4/7411/1 Dashboard: http://10.1.0.4:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [3]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [4]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [5]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [6]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,980,Bob,0.967394992380296,0.28056252081349187
2000-01-01 00:00:01,974,Victor,-0.17653756485871397,0.21702768493235047
2000-01-01 00:00:02,918,Michael,0.558680160455888,0.3947466596832858
2000-01-01 00:00:03,981,Hannah,-0.04913749642414067,0.8963344097843486
2000-01-01 00:00:04,1025,Charlie,0.2635730201814117,-0.7408174351557426
2000-01-01 00:00:05,1035,Yvonne,-0.8353730908967099,-0.055320955767457436
2000-01-01 00:00:06,1034,Victor,0.7929929444891508,-0.04479656537357046
2000-01-01 00:00:07,977,Oliver,-0.16242776864202302,-0.31780866680150166
2000-01-01 00:00:08,1005,Ingrid,0.8602348470097452,-0.9716793924822504


In [7]:
!head data/2000-01-30.csv

timestamp,id,name,x,y
2000-01-30 00:00:00,1031,Jerry,0.20846874845901886,-0.6700753643432995
2000-01-30 00:00:01,1010,Frank,-0.25927073861093297,-0.08508541297100836
2000-01-30 00:00:02,988,Dan,0.05527277499125205,-0.45437252257404137
2000-01-30 00:00:03,1014,Charlie,0.6869155034985899,0.00369272437004331
2000-01-30 00:00:04,986,Ray,0.7176749698597742,0.09141536369906778
2000-01-30 00:00:05,996,Wendy,-0.61345029777449,-0.2742010085768112
2000-01-30 00:00:06,957,Ray,0.15568631078771022,-0.9548618557341502
2000-01-30 00:00:07,1016,Ingrid,-0.7967129192310529,-0.8941642505166083
2000-01-30 00:00:08,1011,Kevin,-0.30565899475775904,0.7471062568910216


We can read one file with `pandas.read_csv` or many files with `dask.dataframe.read_csv`

In [8]:
import pandas as pd

df = pd.read_csv('data/2000-01-01.csv')
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,980,Bob,0.967395,0.280563
1,2000-01-01 00:00:01,974,Victor,-0.176538,0.217028
2,2000-01-01 00:00:02,918,Michael,0.558680,0.394747
3,2000-01-01 00:00:03,981,Hannah,-0.049137,0.896334
4,2000-01-01 00:00:04,1025,Charlie,0.263573,-0.740817


In [9]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,object,int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [10]:
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,980,Bob,0.967395,0.280563
1,2000-01-01 00:00:01,974,Victor,-0.176538,0.217028
2,2000-01-01 00:00:02,918,Michael,0.558680,0.394747
3,2000-01-01 00:00:03,981,Hannah,-0.049137,0.896334
4,2000-01-01 00:00:04,1025,Charlie,0.263573,-0.740817


## Tuning read_csv

The Pandas `read_csv` function has *many* options to help you parse files.  The Dask version uses the Pandas function internally, and so supports many of the same options.  You can use the `?` operator to see the full documentation string.

In [11]:
pd.read_csv?

In [12]:
dd.read_csv?

In this case we use the `parse_dates` keyword to parse the timestamp column to be a datetime.  This will make things more efficient in the future.  Notice that the dtype of the timestamp column has changed from `object` to `datetime64[ns]`.

In [13]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


## Do a simple computation

Whenever we operate on our dataframe we read through all of our CSV data so that we don't fill up RAM.  This is very efficient for memory use, but reading through all of the CSV files every time can be slow.

In [14]:
%time df.groupby('name').x.mean().compute()

CPU times: user 3.73 s, sys: 472 ms, total: 4.21 s
Wall time: 3.79 s


name
Alice      -0.002576
Bob        -0.000098
Charlie    -0.000048
Dan        -0.002654
Edith       0.002538
Frank       0.002125
George     -0.002183
Hannah     -0.002614
Ingrid     -0.000529
Jerry      -0.002148
Kevin      -0.000237
Laura      -0.001633
Michael     0.000894
Norbert     0.000137
Oliver     -0.001553
Patricia   -0.001653
Quinn      -0.002594
Ray        -0.001349
Sarah       0.000501
Tim        -0.000796
Ursula      0.002011
Victor     -0.002205
Wendy      -0.000079
Xavier      0.001953
Yvonne      0.000893
Zelda      -0.001066
Name: x, dtype: float64

## Write to Parquet

Instead, we'll store our data in Parquet, a format that is more efficient for computers to read and write.

In [15]:
df.to_parquet('data/2000-01.parquet', engine='pyarrow')

In [16]:
!ls data/2000-01.parquet/

_common_metadata  part.14.parquet  part.21.parquet  part.29.parquet
_metadata	  part.15.parquet  part.22.parquet  part.3.parquet
part.0.parquet	  part.16.parquet  part.23.parquet  part.4.parquet
part.1.parquet	  part.17.parquet  part.24.parquet  part.5.parquet
part.10.parquet   part.18.parquet  part.25.parquet  part.6.parquet
part.11.parquet   part.19.parquet  part.26.parquet  part.7.parquet
part.12.parquet   part.2.parquet   part.27.parquet  part.8.parquet
part.13.parquet   part.20.parquet  part.28.parquet  part.9.parquet


## Read from Parquet

In [17]:
df = dd.read_parquet('data/2000-01.parquet', engine='pyarrow')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [18]:
%time df.groupby('name').x.mean().compute()

CPU times: user 1.22 s, sys: 90.4 ms, total: 1.31 s
Wall time: 1.27 s


name
Alice      -0.002576
Bob        -0.000098
Charlie    -0.000048
Dan        -0.002654
Edith       0.002538
Frank       0.002125
George     -0.002183
Hannah     -0.002614
Ingrid     -0.000529
Jerry      -0.002148
Kevin      -0.000237
Laura      -0.001633
Michael     0.000894
Norbert     0.000137
Oliver     -0.001553
Patricia   -0.001653
Quinn      -0.002594
Ray        -0.001349
Sarah       0.000501
Tim        -0.000796
Ursula      0.002011
Victor     -0.002205
Wendy      -0.000079
Xavier      0.001953
Yvonne      0.000893
Zelda      -0.001066
Name: x, dtype: float64

## Select only the columns that you plan to use

Parquet is a column-store, which means that it can efficiently pull out only a few columns from your dataset.  This is good because it helps to avoid unnecessary data loading.

In [19]:
%%time
df = dd.read_parquet('data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

CPU times: user 1 s, sys: 58.9 ms, total: 1.06 s
Wall time: 990 ms


name
Alice      -0.002576
Bob        -0.000098
Charlie    -0.000048
Dan        -0.002654
Edith       0.002538
Frank       0.002125
George     -0.002183
Hannah     -0.002614
Ingrid     -0.000529
Jerry      -0.002148
Kevin      -0.000237
Laura      -0.001633
Michael     0.000894
Norbert     0.000137
Oliver     -0.001553
Patricia   -0.001653
Quinn      -0.002594
Ray        -0.001349
Sarah       0.000501
Tim        -0.000796
Ursula      0.002011
Victor     -0.002205
Wendy      -0.000079
Xavier      0.001953
Yvonne      0.000893
Zelda      -0.001066
Name: x, dtype: float64

Here the difference is not that large, but with larger datasets this can save a great deal of time.

## Learn more

http://dask.pydata.org/en/latest/dataframe-create.html